In [1]:
import os

import nbimporter

import utils.file_manager as fm
import utils.pre_processing as ppc

import minetext.clustering.distance as distance
import minetext.clustering.kmedoids as kmedoids

Importing Jupyter notebook from C:\Users\Caio\Desktop\android-comments-miner\utils\file_manager.ipynb
Importing Jupyter notebook from C:\Users\Caio\Desktop\android-comments-miner\utils\pre_processing.ipynb


In [2]:
data_basedir = "C:\\Users\\Caio\\Desktop\\android-comments-miner\\data\\"

exports_basedir = os.path.join(data_basedir, "exports")
clean_exports_basedir = os.path.join(data_basedir, "clean_exports")
results_basedir = os.path.join(data_basedir, "results")
         
# Lista dos nomes dos diretórios que contém os exports
exports_directories = ["facebook", "telegram"]

# Colunas dos exports a serem consideradas
columns = ["Content", "CleanContent", "Source", "Rating"]

distance_calculator = distance.JaccardCalculatorDistance()

In [3]:
for directory in exports_directories:
    filename = directory + ".csv"
    
    export_dirpath = os.path.join(exports_basedir, directory)
    
    # Lê os exports do diretório
    export = fm.read_csv_directory(dirpath=export_dirpath, header=1)
    
    # Realiza o pré-processamento do conteúdo dos exports
    clean_export = ppc.clean_reviews(export=export)
    
    # Salva os exports já pré-processados em um arquivo .csv
    fm.data_to_csv(data=clean_export,\
                   dirpath=clean_exports_basedir,\
                   filename=filename,\
                   columns=columns)

In [5]:
clean_exports = dict()

for directory in exports_directories:
    filename = directory + ".csv"
    
    clean_export = fm.read_csv_file(dirpath=clean_exports_basedir,\
                                   filename=filename)
    
    reviews = clean_export.itertuples()
    
    documents = dict()
    documents["documents"] = []
    
    for export in reviews:
        review = dict()
        
        review["Content"] = export.Content
        review["CleanContent"] = export.CleanContent
        review['Source'] = export.Source
        review["Rating"] = export.Rating
        
        documents["documents"].append(review)
        
    results = dict()
    results["k"] = []
    results["iteractions"] = []
    results["sse"] = []
        
    for i in range(2, 21):
        for j in range(1, 6):
            kmedoids_instance = kmedoids.Kmedoids(k=i,\
                                              documents=documents["documents"],\
                                              distance_calculator=distance_calculator,\
                                              collection_field="documents", text_field_name="CleanContent")
        
            kmedoids_instance.clustering()
            
            results["k"].append(i)
            results["sse"].append(kmedoids_instance.calculate_sse())
            results["iteractions"].append(j)
    
    dirpath = os.path.join(results_basedir, directory)
    
    fm.overall_clustering_results_to_csv(results=results, dirpath=dirpath)